In [1]:
import sys
sys.path.append('../../cloned_model/diff_adapter')
sys.path.append('../')

import os
import argparse
import lightning.pytorch as pl
from lightning.pytorch.strategies import DDPStrategy
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

from util import *
from VIT import build_diff

/Users/hahmwj/anaconda3/envs/hahm/lib/python3.11/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/Users/hahmwj/anaconda3/envs/hahm/lib/python3.11/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/Users/hahmwj/anaconda3/envs/hahm/lib/python3.11/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functiona

In [2]:
dataset = 'hmdb'
batch_size = 2
num_frames = 8
dataloader_train, dataloader_val, dataloader_test, num_classes = load_data(dataset_name = dataset, 
            path = '/Volumes/T7_Shiled/Video_dataset/csv_files', 
            batch_size = batch_size,
            num_workers = 8,
            num_frames = num_frames,
            video_size=224)

Train datasets lengths: 4329
Valid datasets lengths: 1083
Test datasets lengths: 1354


In [3]:
layers = 4
inter_type = 'DM'
differ_block = get_differ_block_layer(layers)
model = build_diff(num_classes = num_classes, 
                    inter_type = inter_type, 
                    differ_layer = differ_block, 
                    num_frames = num_frames,
                    classification_rule = 'concat_fcls_mean_diff', # mean, concat, RNN, LSTM
                    substitute_frame = 1)
# concat_fcls_mean_diff은 substitute_frame은 1이여야만 한다

In [4]:
def convert_weights_to_float(model):
    for param in model.parameters():
        param.data = param.data.float()

convert_weights_to_float(model)

In [5]:
for video, label in dataloader_test:
    output = model(video)[0]
    print(output.size())
    break

cls_tokens:   torch.Size([1, 4, 2, 768])
torch.Size([1, 768])
mean_diff:    torch.Size([1, 768])
torch.Size([1, 51])
